In [11]:
import sys
import matplotlib.colors as colors
import matplotlib.pyplot as plt
sys.path.insert(0,'..')
from nrel_xstock.database import XStockDatabase

In [33]:
database_filepath = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/data/nrel/database.db'
database = XStockDatabase(filepath=database_filepath,apply_changes=True)

In [67]:
# Austin buildings
query = """
SELECT 
    d.dataset_type,
    d.weather_data,
    d.year_of_publication,
    d.release,
    m.bldg_id,
    m.upgrade
FROM metadata m
INNER JOIN dataset d ON 
    d.id = m.dataset_id
INNER JOIN weather w ON 
    w.dataset_id = d.id
    AND w.weather_file_tmy3 = m.in_weather_file_tmy3
WHERE
    m.in_geometry_building_type_recs = 'Single-Family Detached'
    AND m.in_vacancy_status = 'Occupied'
    AND m.in_resstock_county_id = 'TX, Travis County'
    AND w.epw_url IS NOT NULL
ORDER BY
    m.in_pv_system_size ASC,
    m.bldg_id ASC
LIMIT 100
"""
data = database.query_table(query)
data.to_csv('/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/citylearn/CityLearn/nrel_xstock/data/neighborhoods/austin_neighborhood.csv',index=False)
display(data.head())

,dataset_type,weather_data,year_of_publication,release,bldg_id,upgrade
0,resstock,tmy3,2021,1,446460,0
1,resstock,tmy3,2021,1,378790,0
2,resstock,tmy3,2021,1,404350,0
3,resstock,tmy3,2021,1,191067,0
4,resstock,tmy3,2021,1,340590,0


In [62]:
# CityLearn environment input
columns = [
    'Indoor Temperature (C)',
    'Average Unmet Cooling Setpoint Difference (C)',
    'Average Unmet Heating Setpoint Difference (C)',
    'Indoor Relative Humidity (%)',
    'Equipment Electric Power (kWh)',
    'DHW Heating (kWh)',
    'Cooling Load (kWh)',
    'Heating Load (kWh)',
]
buildings = database.query_table("SELECT DISTINCT metadata_id FROM citylearn_energyplus_simulation_result")['metadata_id'].tolist()
cell_height = 0.175
fig_width = cell_height*len(buildings)*1.5
cb_fraction = 0.03
cb_pad = 0.02
figsize = (fig_width,cell_height*len(buildings)*len(columns))
fig, axs = plt.subplots(len(columns),1,figsize=figsize)
divnorm = colors.TwoSlopeNorm(vcenter=0)

for ax, column in zip(fig.axes,columns):
    query = f"""
    SELECT
        ROW_NUMBER() OVER(PARTITION BY m.bldg_id ORDER BY r.Month ASC, r."Day" ASC, r.Hour ASC) AS timestep,
        m.bldg_id,
        r."{column}"
    FROM citylearn_energyplus_simulation_result r
    LEFT JOIN metadata m ON m.id = r.metadata_id
    ORDER BY
        CAST(m.bldg_id AS INTEGER)
    """
    data = database.query_table(query)
    plot_data = data.pivot(index='bldg_id',columns='timestep',values=column)
    plot_data.index = plot_data.index.astype(str)
    x = plot_data.columns
    y = plot_data.index
    z = plot_data.values
    pcm = ax.pcolormesh(x,y,z,shading='nearest',cmap='Blues',norm=divnorm)
    cb = fig.colorbar(pcm,ax=ax,fraction=cb_fraction,pad=cb_pad,label=column)
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Building ID')

plt.tight_layout()
plt.savefig('../data/neighborhoods/austin/austin_neighborhood_heatmap.png',bbox_inches='tight',facecolor='white',)
plt.close()

In [76]:
metadata = database.download_metadata('resstock','tmy3','2021','1')

In [78]:
import pandas as pd
# metadata['in.pv_system_size'] = pd.to_numeric(metadata['in.pv_system_size'],errors='coerce')
metadata[
    (metadata['in.resstock_county_id']=='TX, Travis County')
    &(metadata['in.geometry_building_type_recs']=='Single-Family Detached')
    &(metadata['in.vacancy_status']=='Occupied')
    &(metadata['in.pv_system_size']!='None')
].shape

(23, 248)